In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv("../artifacts/train.csv")

In [3]:
data.head()

,weekday,driver_age,driver_sex,educational_level,driver_relation,driving_exp,vehicle_type,vehicle_owner,service_year,vehicle_defect,...,vehicle_movement,casualty_class,casualty_sex,casualty_age,casualty_severity,casualty_work,casualty_fitness,pedestrian_movement,accident_cause,accident_severity
0,Wednesday,Over 51,Male,Writing & reading,Employee,Below 1yr,Special vehicle,Owner,Unknown,No defect,...,Reversing,Passenger,Female,31-50,3,Driver,Normal,Not a Pedestrian,No priority to vehicle,Slight Injury
1,Saturday,Under 18,Male,Junior high school,Employee,Above 10yr,Automobile,Owner,NaN,No defect,...,Going straight,Driver or rider,Male,18-30,3,NaN,NaN,Not a Pedestrian,Changing lane to the right,Slight Injury
2,Wednesday,18-30,Male,Junior high school,Employee,No Licence,Other,Owner,1-2yr,No defect,...,Other,Pedestrian,Male,31-50,3,Driver,Normal,Unknown or other,No distancing,Serious Injury
3,Monday,18-30,Male,Elementary school,Owner,2-5yr,Turbo,Owner,NaN,No defect,...,Going straight,na,na,na,na,Driver,Normal,Not a Pedestrian,Changing lane to the left,Serious Injury
4,Friday,Under 18,Male,Junior high school,Employee,2-5yr,Automobile,Owner,Unknown,No defect,...,Going straight,Driver or rider,Female,31-50,3,Driver,Normal,Not a Pedestrian,No priority to vehicle,Slight Injury


In [4]:
## Divide the dataset into independent and dependent features

X = data.drop(columns=['accident_severity'], axis=1)

y = data['accident_severity']

In [5]:
X.head()

,weekday,driver_age,driver_sex,educational_level,driver_relation,driving_exp,vehicle_type,vehicle_owner,service_year,vehicle_defect,...,casualty,vehicle_movement,casualty_class,casualty_sex,casualty_age,casualty_severity,casualty_work,casualty_fitness,pedestrian_movement,accident_cause
0,Wednesday,Over 51,Male,Writing & reading,Employee,Below 1yr,Special vehicle,Owner,Unknown,No defect,...,2,Reversing,Passenger,Female,31-50,3,Driver,Normal,Not a Pedestrian,No priority to vehicle
1,Saturday,Under 18,Male,Junior high school,Employee,Above 10yr,Automobile,Owner,NaN,No defect,...,2,Going straight,Driver or rider,Male,18-30,3,NaN,NaN,Not a Pedestrian,Changing lane to the right
2,Wednesday,18-30,Male,Junior high school,Employee,No Licence,Other,Owner,1-2yr,No defect,...,2,Other,Pedestrian,Male,31-50,3,Driver,Normal,Unknown or other,No distancing
3,Monday,18-30,Male,Elementary school,Owner,2-5yr,Turbo,Owner,NaN,No defect,...,2,Going straight,na,na,na,na,Driver,Normal,Not a Pedestrian,Changing lane to the left
4,Friday,Under 18,Male,Junior high school,Employee,2-5yr,Automobile,Owner,Unknown,No defect,...,2,Going straight,Driver or rider,Female,31-50,3,Driver,Normal,Not a Pedestrian,No priority to vehicle


In [6]:
y

0        Slight Injury
1        Slight Injury
2       Serious Injury
3       Serious Injury
4        Slight Injury
             ...      
8616     Slight Injury
8617     Slight Injury
8618     Slight Injury
8619     Slight Injury
8620     Slight Injury
Name: accident_severity, Length: 8621, dtype: object

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [8]:
numerical_columns = data.select_dtypes(exclude="object").columns

numerical_columns

Index(['num_of_vehicles', 'casualty'], dtype='object')

In [9]:
categorical_columns = data.select_dtypes(include="object").columns

categorical_columns

Index(['weekday', 'driver_age', 'driver_sex', 'educational_level',
       'driver_relation', 'driving_exp', 'vehicle_type', 'vehicle_owner',
       'service_year', 'vehicle_defect', 'accident_area', 'lanes',
       'road_alignment', 'junction_type', 'road_type', 'road_conditions',
       'light_condition', 'weather_condition', 'collision_type',
       'vehicle_movement', 'casualty_class', 'casualty_sex', 'casualty_age',
       'casualty_severity', 'casualty_work', 'casualty_fitness',
       'pedestrian_movement', 'accident_cause', 'accident_severity'],
      dtype='object')

In [11]:
num_pipeline = Pipeline(
steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline(
steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ordinalencoder", OrdinalEncoder()),
    ("scaler", StandardScaler())
])

In [12]:
preprocessor = ColumnTransformer(
[
    ("num_pipeline", num_pipeline, numerical_columns),
    ("cat_pipeline", cat_pipeline, categorical_columns)
])

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
X_train = preprocessor.fit_transform(X_train)

ValueError: A given column is not a column of the dataframe